Aim: Input video url, call the download video function, call the data scraping function to get the audio and subtitle files, then input the audio into the fine-tuned whisper model and then add the whisper generated subs to the video

<b> We need to import the finetuned whisper model here and pass the audio into it, for now I'm using the subtitle file directly. <b>

In [1]:
import yt_dlp
import os
import re
import subprocess

In [2]:
def clean_title(filename):
    # Remove invalid characters for filenames
    cleaned = re.sub(r'[\\/:\*?"<>|\.]+', '', filename)
    cleaned = re.sub(r'[^\w\-_\.]', '', cleaned)
    return cleaned.strip()

In [3]:
def download_video(url, output_folder='videos'):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    ydl_opts = {
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'), 
    }

    #Download video metadata to access the title and extension
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        
        #Clean the title and store the original extension
        clean_filename = clean_title(info.get('title', 'video'))
        original_extension = info.get('ext', 'mp4') 

        ydl_opts['outtmpl'] = os.path.join(output_folder, f'{clean_filename}.{original_extension}')

        #Download the video with the cleaned title and original extension
        ydl = yt_dlp.YoutubeDL(ydl_opts)
        ydl.download([url])
        
        return f'{clean_filename}.{original_extension}'

In [4]:
#Downloading Lecture 10: The Completeness of the Real Numbers and Basic Properties of Infinite Series
video_url = 'https://www.youtube.com/watch?v=0_w-R_g5lRA&ab_channel=MITOpenCourseWare'
filename = download_video(video_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=0_w-R_g5lRA&ab_channel=MITOpenCourseWare
[youtube] 0_w-R_g5lRA: Downloading webpage
[youtube] 0_w-R_g5lRA: Downloading ios player API JSON
[youtube] 0_w-R_g5lRA: Downloading web creator player API JSON
[youtube] 0_w-R_g5lRA: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=0_w-R_g5lRA&ab_channel=MITOpenCourseWare
[youtube] 0_w-R_g5lRA: Downloading webpage
[youtube] 0_w-R_g5lRA: Downloading ios player API JSON
[youtube] 0_w-R_g5lRA: Downloading web creator player API JSON
[youtube] 0_w-R_g5lRA: Downloading m3u8 information
[info] 0_w-R_g5lRA: Downloading 1 format(s): 137+251
[download] Destination: videos/Lecture10TheCompletenessoftheRealNumbersandBasicPropertiesofInfiniteSeries.f137.mp4
[download] 100% of  544.19MiB in 00:00:11 at 47.56MiB/s    
[download] Destination: videos/Lecture10TheCompletenessoftheRealNumbersandBasicPropertiesofInfiniteSeries.f251.webm
[download] 100% of   53.25MiB in

In [5]:
def add_subtitle_to_video(filename, subtitle_folder='whisper_sub'):
    # Construct the full path to the video file
    video_file = os.path.join('videos', filename)
    
    base_filename = os.path.splitext(filename)[0]
    
    # Construct the expected subtitle (.vtt) file path
    subtitle_file = os.path.join(subtitle_folder, f'{base_filename}.vtt')
    

    if not os.path.exists(subtitle_file):
        print(f"Subtitle file {subtitle_file} not found.")
        return
    
    output_file = os.path.splitext(video_file)[0] + '_with_subs.mp4'
    
    # FFmpeg command to add the subtitle
    command = [
        'ffmpeg',
        '-i', video_file,  
        '-i', subtitle_file,  
        '-c:v', 'copy',  
        '-c:a', 'copy',  
        '-c:s', 'mov_text',  
        '-map', '0', 
        '-map', '1', 
        output_file  
    ]
    
    try:
        subprocess.run(command, check=True)
        print(f"Subtitle added successfully: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error while adding subtitle: {e}")


In [6]:
add_subtitle_to_video(filename)

ffmpeg version N-117234-g8a951ef5e1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable

Subtitle added successfully: videos/Lecture10TheCompletenessoftheRealNumbersandBasicPropertiesofInfiniteSeries_with_subs.mp4


[out#0/mp4 @ 0x7ff29d811640] video:555558KiB audio:52967KiB subtitle:39KiB other streams:0KiB global headers:0KiB muxing overhead: 0.805101%
frame=136088 fps=65861 q=-1.0 Lsize=  613464KiB time=01:15:30.24 bitrate=1109.3kbits/s speed=2.19e+03x    
